In [1]:
#trying to implement ensemble method
#https://datascience.stackexchange.com/questions/27169/taking-average-of-multiple-neural-networks
#mixture of experts 'with kmeans'
#https://en.wikipedia.org/wiki/Mixture_of_experts
#combining models together university of Tartu
#https://courses.cs.ut.ee/MTAT.03.277/2014_fall/uploads/Main/deep-learning-lecture-9-combining-multiple-neural-networks-to-improve-generalization-andres-viikmaa.pdf
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
#import keras
#from keras.utils import to_categorical

(trX, trY), (teX, teY) = tf.keras.datasets.fashion_mnist.load_data()
trX = trX.reshape(60000, 784)
teX = teX.reshape(10000, 784)

#trY = to_categorical(trY)
#teY = to_categorical(teY)

#np.save('one_hot_train_labels.npy', trY)
#np.save('one_hot_test_labels.npy', teY)

trY = np.load('one_hot_train_labels.npy')
teY = np.load('one_hot_test_labels.npy')


print("x_train shape:", trX.shape, "y_train shape:", trY.shape)
print("x_test shape:", teX.shape, "y_test shape:", teY.shape)

x_train shape: (60000, 784) y_train shape: (60000, 10)
x_test shape: (10000, 784) y_test shape: (10000, 10)


In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w_h1, w_h2, w_o):
    h1 = tf.nn.sigmoid(tf.matmul(X, w_h1)) # this is a basic mlp, think 2 stacked logistic regressions
    h = tf.nn.sigmoid(tf.matmul(h1, w_h2))
    #return tf.matmul(h, w_o, name="insertname_here") if we need to use names and save the models
    return tf.matmul(h, w_o) # note that we dont take the softmax at the end because our cost fn does that for us

#taken from https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def myRange(start,end,step):
    i = start
    while i < end:
        yield i
        i += step
    yield end

def confidence_interval(data):
    #using 95 percent confidence interval
    data_points = len(data)
    average_accuracy = 0
    for i in range(0,data_points):
        average_accuracy = average_accuracy + data[i]/data_points

    print("mean accuracy across k folds: " + str(average_accuracy))

    standard_deviation = 0
    variance = 0
    for i in range(0,data_points):
        variance = variance + ((data[i] - average_accuracy) * (data[i] - average_accuracy))
    
    standard_deviation = np.sqrt(variance/data_points)
    standard_error = standard_deviation/np.sqrt(data_points)

    print("confidence interval: " + str(average_accuracy - 1.96*standard_error) + ", " + str(average_accuracy + 1.96*standard_error))
    return [average_accuracy - 1.96*standard_error, average_accuracy, average_accuracy + 1.96*standard_error]

In [4]:
total_data_set = np.vstack((trX, teX))
total_label_set = np.vstack((trY, teY))

prediction = 0

average_model_accuracy = 0

models_to_train = 5

epochs_per_model = 1

batch_size = 128

number_of_folds = 10

average_model_accuracies = []
ensemble_method_accuracies = []

models_to_train_begin = 2
models_to_train_end = 4

confidence_interval_across_experiments = []

#saver = tf.train.Saver()

# Launch the graph in a session
with tf.Session() as sess:
    for n in range(models_to_train_begin, models_to_train_end + 1):
        models_to_train = n
        average_model_accuracies = []
        ensemble_method_accuracies = []
        for fold in range(0, number_of_folds):
            print("fold number: " + str(fold))
            size_of_fold = int(len(total_data_set)/number_of_folds)
            trX_for_models = np.vstack((total_data_set[0:fold*size_of_fold], total_data_set[(fold + 1)*size_of_fold:len(total_data_set)]))
            teX = total_data_set[fold*size_of_fold:(fold + 1)*size_of_fold]

            trY_for_models = np.vstack((total_label_set[0:fold*size_of_fold], total_label_set[(fold + 1)*size_of_fold:len(total_label_set)]))
            teY = total_label_set[fold*size_of_fold:(fold + 1)*size_of_fold]

            print("x_train shape:", trX.shape, "y_train shape:", trY.shape)
            print("x_test shape:", teX.shape, "y_test shape:", teY.shape)

            average_model_accuracy = 0

            for z in range(0,models_to_train):

                training_per_model = int(len(trX_for_models)/models_to_train)
                trX = trX_for_models[z*training_per_model:(z+1)*training_per_model]
                trY = trY_for_models[z*training_per_model:(z+1)*training_per_model]
                #print(trX.shape)
                #print(trY.shape)

                size_h1 = tf.constant(625, dtype=tf.int32)
                size_h2 = tf.constant(300, dtype=tf.int32)

                X = tf.placeholder("float", [None, 784])
                Y = tf.placeholder("float", [None, 10])

                w_h1 = init_weights([784, size_h1]) # create symbolic variables
                w_h2 = init_weights([size_h1, size_h2])
                w_o = init_weights([size_h2, 10])

                py_x = model(X, w_h1, w_h2, w_o)

                cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)) # compute costs
                train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer
                predict_op = tf.argmax(py_x, 1)
                tf.global_variables_initializer().run()
                for i in range(epochs_per_model):
                    for start, end in zip(myRange(0, len(trX), batch_size), myRange(batch_size, len(trX)+1, batch_size)):
                        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
                    print(i, np.mean(np.argmax(teY, axis=1) ==
                                     sess.run(predict_op, feed_dict={X: teX})))
                average_model_accuracy = np.mean(np.argmax(teY, axis=1) == sess.run(predict_op, feed_dict={X: teX}))/models_to_train + average_model_accuracy
                if z == 0:
                    prediction = sess.run(py_x, feed_dict={X: teX})
                else:
                    prediction = np.add(prediction, sess.run(py_x, feed_dict={X: teX}))

            print("average accuracy of all the models: " + str(average_model_accuracy))
            print("accuracy of the predictions of all the models combined: " + str(np.mean(np.argmax(teY, axis=1) == np.argmax(prediction, axis=1)))) 
            average_model_accuracies.append(average_model_accuracy)
            ensemble_method_accuracies.append(np.mean(np.argmax(teY, axis=1) == np.argmax(prediction, axis=1)))

        answer = confidence_interval(ensemble_method_accuracies)
        print(answer)
        confidence_interval_across_experiments.append(answer)

print(confidence_interval_across_experiments)
confidence_interval_across_experiments = np.asarray(confidence_interval_across_experiments)
np.save("ensembleMethodResults.npy", confidence_interval_across_experiments)
    
        #saver.save(sess,"mlp/session.ckpt")

fold number: 0
x_train shape: (60000, 784) y_train shape: (60000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

0 0.46385714285714286
0 0.4997142857142857
average accuracy of all the models: 0.48178571428571426
accuracy of the predictions of all the models combined: 0.5105714285714286
fold number: 1
x_train shape: (31500, 784) y_train shape: (31500, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.4561428571428571
0 0.5007142857142857
average accuracy of all the models: 0.4784285714285714
accuracy of the predictions of all the models combined: 0.5034285714285714
fold number: 2
x_train shape: (31500, 784) y_train shape: (31500, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.46314285714285713
0 0.488
average accuracy of all the models: 0.47557142857142853
accuracy of t

average accuracy of all the models: 0.258
accuracy of the predictions of all the models combined: 0.269
fold number: 6
x_train shape: (15750, 784) y_train shape: (15750, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.19857142857142857
0 0.28714285714285714
0 0.24257142857142858
0 0.30114285714285716
average accuracy of all the models: 0.25735714285714284
accuracy of the predictions of all the models combined: 0.268
fold number: 7
x_train shape: (15750, 784) y_train shape: (15750, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.198
0 0.2805714285714286
0 0.20042857142857143
0 0.2937142857142857
average accuracy of all the models: 0.24317857142857144
accuracy of the predictions of all the models combined: 0.2402857142857143
fold number: 8
x_train shape: (15750, 784) y_train shape: (15750, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.1997142857142857
0 0.2862857142857143
0 0.19457142857142856
0 0.2957142857142857
average accuracy of all the models: 0.2

average accuracy of all the models: 0.2053809523809524
accuracy of the predictions of all the models combined: 0.18671428571428572
fold number: 9
x_train shape: (10500, 784) y_train shape: (10500, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.09657142857142857
0 0.2732857142857143
0 0.17714285714285713
0 0.26485714285714285
0 0.304
0 0.2557142857142857
average accuracy of all the models: 0.2285952380952381
accuracy of the predictions of all the models combined: 0.2814285714285714
fold number: 0
x_train shape: (10500, 784) y_train shape: (10500, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.09914285714285714
0 0.3922857142857143
0 0.203
0 0.09857142857142857
0 0.18371428571428572
0 0.35285714285714287
0 0.278
average accuracy of all the models: 0.2296530612244898
accuracy of the predictions of all the models combined: 0.447
fold number: 1
x_train shape: (9000, 784) y_train shape: (9000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.179
0 0.27871428

0 0.20114285714285715
0 0.22442857142857142
0 0.2815714285714286
0 0.2694285714285714
0 0.22242857142857142
average accuracy of all the models: 0.223125
accuracy of the predictions of all the models combined: 0.3127142857142857
fold number: 0
x_train shape: (7875, 784) y_train shape: (7875, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.10114285714285715
0 0.2735714285714286
0 0.21185714285714285
0 0.293
0 0.18857142857142858
0 0.2745714285714286
0 0.10114285714285715
0 0.19428571428571428
0 0.2012857142857143
average accuracy of all the models: 0.20438095238095239
accuracy of the predictions of all the models combined: 0.20114285714285715
fold number: 1
x_train shape: (7000, 784) y_train shape: (7000, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.27014285714285713
0 0.266
0 0.23314285714285715
0 0.284
0 0.19614285714285715
0 0.22957142857142857
0 0.10342857142857143
0 0.19228571428571428
0 0.19585714285714287
average accuracy of all the models: 0.2189523809523

0 0.31085714285714283
0 0.19242857142857142
0 0.098
0 0.102
0 0.10057142857142858
0 0.19614285714285715
0 0.09371428571428571
0 0.10057142857142858
0 0.10057142857142858
0 0.19628571428571429
average accuracy of all the models: 0.14911428571428573
accuracy of the predictions of all the models combined: 0.2422857142857143
fold number: 9
x_train shape: (6300, 784) y_train shape: (6300, 10)
x_test shape: (7000, 784) y_test shape: (7000, 10)
0 0.3177142857142857
0 0.18685714285714286
0 0.10028571428571428
0 0.10042857142857142
0 0.09971428571428571
0 0.19328571428571428
0 0.101
0 0.09971428571428571
0 0.19271428571428573
0 0.10028571428571428
average accuracy of all the models: 0.1492
accuracy of the predictions of all the models combined: 0.19214285714285714
mean accuracy across k folds: 0.24287142857142857
confidence interval: 0.21815545410955473, 0.2675874030333024
[0.21815545410955473, 0.24287142857142857, 0.2675874030333024]
[[0.21815545410955473, 0.24287142857142857, 0.26758740303330